In [1]:
import glob
import os

import btrack
import numpy as np

from skimage.io import imread

from tqdm import tqdm

In [2]:
from cellx.tools.image import InfinitePaddedImage

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50

import logging

logger = tf.get_logger()
logger.setLevel(logging.WARNING)

In [4]:
PATH = "/media/quantumjot/Data/Manasi/MK0000/"
POSITION = "Position1"
CHANNELS = ("Chan2", )
INPUT_SHAPE = (64, 64, 3)

In [5]:
def segmentation_generator(position):
    files = [
        os.path.join(PATH, position, "cellpose", f"cellpose_{position}_{i}_cp_masks.png") 
        for i in range(150)
    ]
    
    for file in files:
        yield imread(file)

In [6]:
objects = btrack.utils.segmentation_to_objects(
    segmentation_generator(POSITION), 
    properties = ('area', ), 
)

[INFO][2021/08/25 11:17:54 AM] Localizing objects from segmentation...
[INFO][2021/08/25 11:17:55 AM] Objects are of type: <class 'dict'>
[INFO][2021/08/25 11:17:55 AM] ...Found 15446 objects in 150 frames.


In [7]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area
0,0,250.533784,3.141892,0.0,0,False,0,5,0.0,148


In [8]:
# objects = objects[:100]

In [9]:
def feature_generator(position, channel, objects):
    stack = imread(os.path.join(PATH, position, f"{channel}_Concatenated.tif"))
    
    win = INPUT_SHAPE[0] // 2
    
    for obj in tqdm(objects):
        idx = obj.t
        frame = InfinitePaddedImage(stack[idx, ...])
        
        sx = slice(int(obj.x)-win, int(obj.x)+win, 1)
        sy = slice(int(obj.y)-win, int(obj.y)+win, 1)
        
        crop = frame[sy, sx]
        
        yield np.stack([crop]*3, axis=-1)[np.newaxis, ...]

In [10]:
model = ResNet50(
    include_top=False,         
    weights="imagenet",        
    input_shape=INPUT_SHAPE,
    pooling='avg',
)

2021-08-25 11:17:56.495922: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-25 11:17:56.875855: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30049 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:65:00.0, compute capability: 7.0


In [11]:
y = model.predict(feature_generator(POSITION, CHANNELS[0], objects))

  0%|                                                                                           | 1/15446 [00:00<2:39:27,  1.61it/s]2021-08-25 11:18:06.403207: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202
2021-08-25 11:18:06.696809: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
100%|█████████████████████████████████████████████████████████████████████████████████████████| 15446/15446 [04:12<00:00, 61.21it/s]


In [12]:
y.shape

(15446, 2048)

In [13]:
assert y.shape[0] == len(objects)

In [14]:
for i, obj in enumerate(objects):
    obj.properties = {'resnet-features': y[i, ...]}

In [15]:
objects[0]

,ID,x,y,z,t,dummy,states,label,prob,area,resnet-features
0,0,250.533784,3.141892,0.0,0,False,0,5,0.0,148,"(2048,) array"


In [16]:
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('/home/quantumjot/Dropbox/Code/py3/BayesianTracker/models/cell_config.json')
    tracker.max_search_radius = 30

    # append the objects to be tracked
    tracker.append(objects)

    # set the volume
    tracker.volume=((0, 512), (0, 512), (-1e5, 1e5))

    # track them (in interactive mode)
    tracker.track_interactive(step_size=100)

    # generate hypotheses and run the global optimizer
    tracker.optimize()

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_1')
    
    tracks = tracker.tracks

[INFO][2021/08/25 11:22:18 AM] Loaded btrack: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.so
[INFO][2021/08/25 11:22:18 AM] btrack (v0.4.2) library imported
[INFO][2021/08/25 11:22:18 AM] Setting max XYZ search radius to: 100
[INFO][2021/08/25 11:22:18 AM] Starting BayesianTracker session
[INFO][2021/08/25 11:22:18 AM] Loading configuration file: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/models/cell_config.json
[INFO][2021/08/25 11:22:18 AM] Loading motion model: b'cell_motion'
[INFO][2021/08/25 11:22:18 AM] Setting max XYZ search radius to: 30
[INFO][2021/08/25 11:22:18 AM] Objects are of type: <class 'list'>
[INFO][2021/08/25 11:22:18 AM] Set volume to ((0, 512), (0, 512), (-100000.0, 100000.0))
[INFO][2021/08/25 11:22:18 AM] Starting tracking... 
[INFO][2021/08/25 11:22:18 AM] Tracking objects in frames 0 to 99 (of 150)...


Instantiating BTRACK interface wrapper (v0.4.2, compiled Jul 13 2021 at 11:23:01)


[INFO][2021/08/25 11:22:18 AM]  - Timing (Bayesian updates: 2.14ms, Linking: 0.27ms)
[INFO][2021/08/25 11:22:18 AM]  - Probabilities (Link: 0.61232, Lost: 1.00000)
[INFO][2021/08/25 11:22:18 AM]  - Stats (Active: 104, Lost: 339, Conflicts resolved: 218)
[INFO][2021/08/25 11:22:18 AM] Tracking objects in frames 100 to 150 (of 150)...
[INFO][2021/08/25 11:22:18 AM]  - Timing (Bayesian updates: 4.86ms, Linking: 0.43ms)
[INFO][2021/08/25 11:22:18 AM]  - Probabilities (Link: 1.00000, Lost: 1.00000)
[INFO][2021/08/25 11:22:18 AM] SUCCESS.
[INFO][2021/08/25 11:22:18 AM]  - Found 330 tracks in 150 frames (in 0.0s)
[INFO][2021/08/25 11:22:18 AM]  - Inserted 120 dummy objects to fill tracking gaps
[INFO][2021/08/25 11:22:18 AM] Loading hypothesis model: cell_hypothesis
[INFO][2021/08/25 11:22:18 AM] Calculating hypotheses (relax: True)...
[INFO][2021/08/25 11:22:18 AM] Setting up constraints matrix for global optimisation...
[INFO][2021/08/25 11:22:18 AM] Optimizing...
[INFO][2021/08/25 11:22:18

GLPK Integer Optimizer, v4.65
1320 rows, 1182 columns, 1765 non-zeros
1182 integer variables, all of which are binary
Preprocessing...
660 rows, 1182 columns, 1765 non-zeros
1182 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 660
Solving LP relaxation...
GLPK Simplex Optimizer, v4.65
660 rows, 1182 columns, 1765 non-zeros
*     0: obj =   1.875228078e+03 inf =   0.000e+00 (220)
*   163: obj =   7.315112464e+02 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+   163: mip =     not found yet >=              -inf        (1; 0)
+   163: >>>>>   7.315112464e+02 >=   7.315112464e+02   0.0% (1; 0)
+   163: mip =   7.315112464e+02 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


[INFO][2021/08/25 11:22:19 AM] Writing properties/obj_type_1/area (15446,)
[INFO][2021/08/25 11:22:19 AM] Writing properties/obj_type_1/resnet-features (15446, 2048)
[INFO][2021/08/25 11:22:19 AM] Writing tracks/obj_type_1
[INFO][2021/08/25 11:22:19 AM] Writing dummies/obj_type_1
[INFO][2021/08/25 11:22:19 AM] Writing LBEP/obj_type_1
[INFO][2021/08/25 11:22:19 AM] Writing fates/obj_type_1
[INFO][2021/08/25 11:22:19 AM] Closing HDF file: /media/quantumjot/Data/Manasi/MK0000/tracking.h5
[INFO][2021/08/25 11:22:19 AM] Ending BayesianTracker session


Deleting BTRACK interface wrapper


In [19]:
tracks[1]

,ID,t,x,y,z,parent,root,state,generation,dummy,area,resnet-features
0,50,0,204.826087,5.821510,0.0,50,50,5,0,False,437,"(2048,) array"
1,50,1,208.464968,6.507431,0.0,50,50,5,0,False,471,"(2048,) array"
2,50,2,214.723404,4.885638,0.0,50,50,5,0,False,376,"(2048,) array"
